In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import string
from nltk.corpus import stopwords
from emot.emo_unicode import UNICODE_EMO, EMOTICONS
from string import digits
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras import layers
from keras.layers import Activation, Dense , Dropout,Bidirectional, GlobalMaxPool1D , LSTM
from keras.layers import Embedding, LSTM, Dense, Dropout,Conv1D,GlobalMaxPooling1D
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.utils import to_categorical
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding
from sklearn.metrics import roc_auc_score
from nltk.tokenize import word_tokenize
import gensim
from sklearn.metrics import classification_report,confusion_matrix

Using TensorFlow backend.


In [2]:
forumPosts = pd.read_excel('stanfordMOOCForumPostsSet.xlsx')

In [3]:
forumPosts[:5]

,Text,Opinion(1/0),Question(1/0),Answer(1/0),Sentiment(1-7),Confusion(1-7),Urgency(1-7),CourseType,forum_post_id,course_display_name,forum_uid,created_at,post_type,anonymous,anonymous_to_peers,up_count,comment_thread_id,reads
0,Interesting! How often we say those things to ...,1,0,0,6.5,2.0,1.5,Education,5225177f2c501f0a00000015,Education/EDUC115N/How_to_Learn_Math,30CADB93E6DE4711193D7BD05F2AE95C,2013-09-02 22:55:59,Comment,0.0,0.0,0.0,5221a8262cfae31200000001,41.0
1,"What is \Algebra as a Math Game\"" or are you j...",0,1,0,4.0,5.0,3.5,Education,5207d0e9935dfc0e0000005e,Education/EDUC115N/How_to_Learn_Math,37D8FAEE7D0B94B6CFC57D98FD3D0BA5,2013-08-11 17:59:05,Comment,0.0,0.0,0.0,520663839df35b0a00000043,55.0
2,I like the idea of my kids principal who says ...,1,0,0,5.5,3.0,2.5,Education,52052c82d01fec0a00000071,Education/EDUC115N/How_to_Learn_Math,CC11480215042B3EB6E5905EAB13B733,2013-08-09 17:53:06,Comment,0.0,0.0,0.0,51e59415e339d716000001a6,25.0
3,"From their responses, it seems the students re...",1,0,0,6.0,3.0,2.5,Education,5240a45e067ebf1200000008,Education/EDUC115N/How_to_Learn_Math,C717F838D10E8256D7C88B33C43623F1,2013-09-23 20:28:14,CommentThread,0.0,0.0,0.0,None,0.0
4,"The boys loved math, because \there is freedom...",1,0,0,7.0,2.0,3.0,Education,5212c5e2dd10251500000062,Education/EDUC115N/How_to_Learn_Math,F83887D68EA48964687C6441782CDD0E,2013-08-20 01:26:58,CommentThread,0.0,0.0,0.0,None,3.0


### The below Cell contains all the Functions used in the Notebook which help us in pre processing the Data.

* remove_punct : This function is used to remove all the punctuations from the text
* tokenization : This function is used to split longer strings of data into smaller strings
* load_data : In this fucntion, we are converting the data which we obtained into readable format, in this case into dataframes. We also remove jargon values from the text such as start, end and user.
* stemming : We use stemming to remove the affixes from a word and obtain the root word
* lemmatizer : We use lemmatization to capture canonical forms based on a word's lemma. Eg : better → good
* convert_emojis : We use the convert emojis fucntion to convert the emojis into the their meaning. eg : a sushi emoji will be changed to the word sushi.
* convert_emoticons : We use the convert emojis fucntion to convert the emojis into the their meaning. eg : a happy emoji will be changed to the text happy.
* preprocessing : The preprocessing function is used to preprocess the text. In this function we call the others functions too which will help us in preprocessing the data. We remove punctuations, we remove stop words, emojis and emoticons. We also sem and lemmatize the data. we change the data into lower case.

In [4]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

def tokenization(text):
    text = re.split('\W+', text)
    return text

ps = nltk.PorterStemmer()

def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

wn = nltk.WordNetLemmatizer()

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text

# Converting emojis to words
def convert_emojis(text):
    for emot in UNICODE_EMO:
        text = text.replace(emot, "_".join(UNICODE_EMO[emot].replace(",","").replace(":","").split()))
    return text

# Converting emoticons to words    
def convert_emoticons(text):
    for emot in EMOTICONS:
        text = re.sub(u'('+emot+')', "_".join(EMOTICONS[emot].replace(",","").split()), text)
    return text

def preprocessing(df):    
    df= df.apply(lambda x: convert_emojis(x))
    df= df.apply(lambda x: convert_emoticons(x))
    df = df.apply(lambda x: " ".join(x.lower() for x in x.split()))
    stop = stopwords.words('english')
    df = df.apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    df = df.apply(lambda x: remove_punct(x))
    df = df.apply(lambda x: tokenization(x.lower()))
    df = df.apply(lambda x: stemming(x))
    df = df.apply(lambda x: lemmatizer(x))
    
    for i in range(0, len(df)):
        processed_feature = re.sub(r'\W', ' ', str(df[i]))
        processed_feature = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)
        processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)
        processed_feature = re.sub(r'^b\s+', '', processed_feature)
        df[i] = processed_feature.lower()
    return df

#### While preprocessing the texts, we were facing issues because all the forumPosts['Text'] values were not in the string format and were in Int and Float format too. 

#### Hence, we converted all the values to the String format and then proceeded with preprocessing 

In [5]:
print("Index of Values which are not string type")
for i in range(0, len(forumPosts)):
    #print(type(forumPosts['Text'][i]))
    
    if type(forumPosts['Text'][i])!=str:
        print(i)

Index of Values which are not string type
11157
18312
19732
23525
24285
27323


In [6]:
for i in range(0, len(forumPosts)):
    if type(forumPosts['Text'][i])!=str:
        forumPosts['Text'][i] = str(forumPosts['Text'][i])


C:\Users\abhis\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
print("Index of Values which are not string type")
for i in range(0, len(forumPosts)):
    #print(type(forumPosts['Text'][i]))
    if type(forumPosts['Text'][i])!=str:
        print(i)

Index of Values which are not string type


In [8]:
forumPosts["Text"] = preprocessing(forumPosts["Text"])

In [9]:
X = forumPosts.drop('Sentiment(1-7)', axis=1)
y = forumPosts['Sentiment(1-7)']

In [10]:
# 80% training and 20% test
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=69) 

In [11]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(23683, 17)
(5921, 17)
(23683,)
(5921,)


# Vectorize using TFIDF

In [12]:
tfidf_vect = TfidfVectorizer(max_features=500,min_df=5, max_df=0.8,ngram_range=(1,3))
tfidf_vect.fit(X_train["Text"])
X_train_tfidf = tfidf_vect.transform(X_train["Text"])
X_test_tfidf = tfidf_vect.transform(X_test["Text"])
#y_train_dummy = to_categorical(y_train)
#y_test_dummy = to_categorical(y_test)
y_train_dummy = pd.get_dummies(y_train).values
y_test_dummy = pd.get_dummies(y_test).values

In [13]:
print(X_train_tfidf.shape)
print(X_test_tfidf.shape)
print(y_train_dummy.shape)
print(y_test_dummy.shape)

(23683, 500)
(5921, 500)
(23683, 13)
(5921, 13)


# Building a CNN Model

* Dense is a standard layer type that is used in many cases for neural networks.
* Relu that is rectified linear activation function returns the value provided as input directly, when training a neural network.
* add function is used to add layers to our model.
* Sequential model is used as the layers are stacked sequentially that is input and output layer with their respective shapes.
* As the output layer is a multiclass classification problem "softmax" has been used as output layer.


In [14]:
vocab_size=50000
maxlen = X_train_tfidf.shape[1]

embedding_size=64
model=Sequential()
model.add(Embedding(vocab_size, embedding_size, input_length=maxlen))
model.add(Dropout(0.25))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(13, activation='softmax'))


In [15]:
model.compile(loss='categorical_crossentropy', 
             optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 64)           3200000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 500, 64)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 496, 128)          41088     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
_________________________________________________________________
dropout_2 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 13)               

In [16]:
history = model.fit(X_train_tfidf, y_train_dummy,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test_tfidf, y_test_dummy),
                    batch_size=128)

C:\Users\abhis\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 23683 samples, validate on 5921 samples
Epoch 1/10
23683/23683 [==============================] - 56s 2ms/step - loss: 1.9155 - accuracy: 0.4179 - val_loss: 1.6037 - val_accuracy: 0.5516
Epoch 2/10
23683/23683 [==============================] - 57s 2ms/step - loss: 1.6623 - accuracy: 0.5251 - val_loss: 1.5780 - val_accuracy: 0.5516
Epoch 3/10
23683/23683 [==============================] - 53s 2ms/step - loss: 1.6177 - accuracy: 0.5419 - val_loss: 1.5775 - val_accuracy: 0.5516
Epoch 4/10
23683/23683 [==============================] - 48s 2ms/step - loss: 1.5962 - accuracy: 0.5476 - val_loss: 1.5584 - val_accuracy: 0.5516
Epoch 5/10
23683/23683 [==============================] - 52s 2ms/step - loss: 1.5781 - accuracy: 0.5495 - val_loss: 1.5480 - val_accuracy: 0.5516
Epoch 6/10
23683/23683 [==============================] - 53s 2ms/step - loss: 1.5700 - accuracy: 0.5504 - val_loss: 1.5251 - val_accuracy: 0.5516
Epoch 7/10
23683/23683 [==============================] - 53s 2ms/ste

In [17]:
from sklearn.metrics import classification_report,confusion_matrix

y_pred_tfidf = model.predict(X_test_tfidf)
y_test_dummy = np.argmax(y_test_dummy, axis=1)
y_pred_tfidf = np.argmax(y_pred_tfidf, axis=1)
print(classification_report(y_test_dummy,y_pred_tfidf))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.00      0.00      0.00         9
           2       0.00      0.00      0.00        16
           3       0.00      0.00      0.00        79
           4       0.00      0.00      0.00       246
           5       0.00      0.00      0.00       514
           6       0.55      1.00      0.71      3266
           7       0.00      0.00      0.00       882
           8       0.00      0.00      0.00       464
           9       0.00      0.00      0.00       226
          10       0.00      0.00      0.00       144
          11       0.00      0.00      0.00        55
          12       0.00      0.00      0.00        16

    accuracy                           0.55      5921
   macro avg       0.04      0.08      0.05      5921
weighted avg       0.30      0.55      0.39      5921



C:\Users\abhis\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
pred_prob_tfidf = model.predict_proba(X_test_tfidf)
auc_score = roc_auc_score(y_test_dummy, pred_prob_tfidf, multi_class="ovo",
                                  average="macro")
print('ROC-AUC = %.2f'% auc_score)

ROC-AUC = 0.50


# Word2Vec features

In [19]:
text_lines = list()
texts = forumPosts['Text'].values.tolist()

texts

[' interest often say thing other without realli understand say must power experi excel ',
 ' algebra math game say creat game incorpor algebra ',
 ' like idea kid princip say smart mean easi smart mean work hard incorpor idea make mistak work hard ',
 ' respons seem student realli like power felt free solv math way want use academ languag like decompos number friendli number abl explain mean ',
 ' boy love math there freedom anyth great way see math number think student realiz math could taught differ thought that learn third grader need know go fourth grade thi inspir happyfaceorsmiley ',
 ' pervas idea good math somehow otherwis absentmind geeki geniu cant relat regular peopl ',
 ' peer review modul fulli set yet done anyth wrong professor boaler simpli bit work fulli readi particip you read tech support team say hrefskepticalannoyedundecideduneasyorhesitanthttpsskepticalannoyedundecideduneasyorhesitantclassstanfordeducourseseducationeducnhowtolearnmathdiscussionforumixedxtemplatesc

In [20]:
for text in texts:
    tokens=word_tokenize(text)
    tokens=[w.lower() for w in tokens]
    table = str.maketrans('','',string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not  w in stop_words]
    text_lines.append(words)

In [21]:
len(text_lines)

29604

In [22]:
gensim_model = gensim.models.Word2Vec(sentences=text_lines,
                              min_count=1,
                     window=5,
                     size=100, 
                     workers=4)
words = list(gensim_model.wv.vocab)
print('Vocab Size: %d' % len(words))

Vocab Size: 32838


In [23]:
words[:10]

['interest',
 'often',
 'say',
 'thing',
 'without',
 'realli',
 'understand',
 'must',
 'power',
 'experi']

In [24]:
gensim_model.wv.most_similar('interest')

[('wonder', 0.800317108631134),
 ('latino', 0.7716184258460999),
 ('amaz', 0.7641400694847107),
 ('esoter', 0.7592405080795288),
 ('fascin', 0.7583029270172119),
 ('great', 0.7553567886352539),
 ('excit', 0.7547274231910706),
 ('love', 0.7496294379234314),
 ('awww', 0.7446329593658447),
 ('enjoy', 0.7383421063423157)]

In [25]:
gensim_model['enjoy']

C:\Users\abhis\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-0.14083579,  0.43893436,  0.59562314, -1.2491006 ,  0.11296221,
       -0.4065856 ,  0.18201794,  1.3642101 ,  0.90149456,  0.24203414,
        1.6757509 ,  1.0320046 ,  0.10149748,  1.3847531 , -1.6498253 ,
       -0.47544116, -0.15619004, -1.3283421 ,  0.15382482, -1.1238902 ,
        1.1694499 ,  0.28447324,  0.33180994,  0.6271691 ,  0.13756038,
        0.7365669 , -1.0686493 ,  0.4311466 ,  0.76590973, -2.1141376 ,
        0.09923551, -0.69930744, -2.2000027 , -1.485335  ,  0.97930616,
        0.9820206 , -0.34914467,  0.24002142, -0.17154354, -0.41829613,
        1.0564042 ,  0.29273874,  0.7912744 ,  0.09539467,  1.0451264 ,
       -0.00525214,  1.610031  ,  1.2853937 , -0.02739275, -0.30298406,
       -0.8451619 ,  0.27171314,  0.03981507,  0.49612224,  1.4699256 ,
       -0.03182014, -0.659477  , -0.12576029,  0.7257643 , -0.4076686 ,
       -0.54953676,  0.42941508,  0.3432219 , -0.13789819, -1.0477085 ,
       -1.8065944 , -0.44038826, -0.85177386,  1.0182081 ,  0.45

## The parameters:

* min_count = int - Ignores all words with total absolute frequency lower than this - (2, 100)
* window = int - The maximum distance between the current and predicted word within a sentence. E.g. window words on the left and window words on the left of our target - (2, 10)
* size = int - Dimensionality of the feature vectors. - (50, 300)
* sample = float - The threshold for configuring which higher-frequency words are randomly downsampled. Highly influencial. - (0, 1e-5)
* alpha = float - The initial learning rate - (0.01, 0.05)
* min_alpha = float - Learning rate will linearly drop to min_alpha as training progresses. To set it: alpha - (min_alpha * epochs) ~ 0.00
* negative = int - If > 0, negative sampling will be used, the int for negative specifies how many "noise words" should be drown. If set to 0, no negative sampling is used. - (5, 20)
* workers = int - Use these many worker threads to train the model (=faster training with multicore machines)

In [39]:
filename = 'model.txt'
gensim_model.wv.save_word2vec_format(filename,binary=False)

In [40]:
embedding_index = {}

f = open(os.path.join('','model.txt'),encoding="utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embedding_index[word] = coefs
f.close

<function TextIOWrapper.close()>

In [41]:
tokenized_object = Tokenizer()
tokenized_object.fit_on_texts(text_lines)
seq = tokenized_object.texts_to_sequences(text_lines)

#pad sequence
word_index = tokenized_object.word_index
print('Unique token %d' % len(word_index))
text_pad = pad_sequences(seq,maxlen=268)
sentiment = forumPosts['Sentiment(1-7)']
print(text_pad.shape)
print(sentiment.shape)

Unique token 32838
(29604, 268)
(29604,)


In [42]:
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words,100))

for word,i in word_index.items():
    if i > num_words:
        continue
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


In [43]:
print(num_words)

32839


In [44]:
X_train_word2vec, X_test_word2vec, y_train_word2vec, y_test_word2vec = train_test_split(text_pad,sentiment, test_size=0.2,random_state=109) 

In [45]:
vocab_size=50000
maxlen = X_train_word2vec.shape[1]

embedding_size=64
model_w2v=Sequential()
model_w2v.add(Embedding(vocab_size, embedding_size, input_length=maxlen))
model_w2v.add(Dropout(0.25))
model_w2v.add(Conv1D(128, 5, activation='relu'))
model_w2v.add(GlobalMaxPooling1D())
model_w2v.add(Dense(10, activation='relu'))
model_w2v.add(Dropout(0.25))

model_w2v.add(Dense(13, activation='softmax'))


In [46]:
y_train_word2vec_dummy = pd.get_dummies(y_train_word2vec).values
y_test_word2vec_dummy = pd.get_dummies(y_test_word2vec).values

print(y_train_word2vec_dummy.shape)
print(y_test_word2vec_dummy.shape)

(23683, 13)
(5921, 13)


In [47]:
model_w2v.compile(loss='categorical_crossentropy', 
             optimizer='adam', 
              metrics=['accuracy'])
model_w2v.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 268, 64)           3200000   
_________________________________________________________________
dropout_5 (Dropout)          (None, 268, 64)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 264, 128)          41088     
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1290      
_________________________________________________________________
dropout_6 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 13)               

In [48]:
history = model_w2v.fit(X_train_word2vec, y_train_word2vec_dummy,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test_word2vec, y_test_word2vec_dummy),
                    batch_size=512)

C:\Users\abhis\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 23683 samples, validate on 5921 samples
Epoch 1/10
23683/23683 [==============================] - 25s 1ms/step - loss: 2.1686 - accuracy: 0.3223 - val_loss: 1.8288 - val_accuracy: 0.5626
Epoch 2/10
23683/23683 [==============================] - 25s 1ms/step - loss: 1.8577 - accuracy: 0.4775 - val_loss: 1.6530 - val_accuracy: 0.5626
Epoch 3/10
23683/23683 [==============================] - 25s 1ms/step - loss: 1.6547 - accuracy: 0.5131 - val_loss: 1.4493 - val_accuracy: 0.5732
Epoch 4/10
23683/23683 [==============================] - 29s 1ms/step - loss: 1.4598 - accuracy: 0.5341 - val_loss: 1.2858 - val_accuracy: 0.5850
Epoch 5/10
23683/23683 [==============================] - 29s 1ms/step - loss: 1.3459 - accuracy: 0.5534 - val_loss: 1.2572 - val_accuracy: 0.5862
Epoch 6/10
23683/23683 [==============================] - 26s 1ms/step - loss: 1.2644 - accuracy: 0.5649 - val_loss: 1.2329 - val_accuracy: 0.5828
Epoch 7/10
23683/23683 [==============================] - 28s 1ms/ste

In [49]:
y_pred_word2vec = model_w2v.predict(X_test_word2vec)
y_test_word2vec_dummy = np.argmax(y_test_word2vec_dummy, axis=1)
y_pred_word2vec = np.argmax(y_pred_word2vec, axis=1)
print(classification_report(y_test_word2vec_dummy,y_pred_word2vec))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00        10
           2       0.00      0.00      0.00         8
           3       0.00      0.00      0.00        72
           4       0.00      0.00      0.00       223
           5       0.24      0.14      0.18       525
           6       0.72      0.87      0.78      3331
           7       0.33      0.47      0.39       846
           8       0.24      0.20      0.22       466
           9       0.00      0.00      0.00       242
          10       0.00      0.00      0.00       130
          11       0.00      0.00      0.00        52
          12       0.00      0.00      0.00        15

    accuracy                           0.58      5921
   macro avg       0.12      0.13      0.12      5921
weighted avg       0.49      0.58      0.53      5921



C:\Users\abhis\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [50]:
pred_prob_word2vec = model_w2v.predict_proba(X_test_word2vec)
auc_score_w2v = roc_auc_score(y_test_word2vec_dummy, pred_prob_word2vec, multi_class="ovo",
                                  average="macro")
print('ROC-AUC = %.2f'% auc_score_w2v)

ROC-AUC = 0.74
